### Datasource: http://vision.stanford.edu/aditya86/ImageNetDogs/

Attribution: https://github.com/AIPI540/AIPI540-Deep-Learning-Applications/

Jon Reifschneider
Brinnae Bent 

In [ ]:
import torch
import torchvision
from torchvision import transforms
import matplotlib as plt

# Define the transformation
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load the data
train_data = torchvision.datasets.ImageFolder(root="output/train/", transform=transform)
test_data = torchvision.datasets.ImageFolder(root="output/test/", transform=transform)

# Define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
import os

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import inception_v3
import torch.nn.functional as F


In [ ]:
# Define the model
model = inception_v3(pretrained=True)

# Replace the last layer
num_features = model.fc.in_features
   
class Multiclass_Net(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_hidden3, n_output):
        super().__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden1)
        self.hidden2 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden3 = nn.Linear(n_hidden2, n_hidden3)
        self.out = nn.Linear(n_hidden3, n_output)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.out(x)
        return x

# Instantiate our neural network
# n_input=4 since we have 4 features
# n_output=3 since we have 3 classes
net = Multiclass_Net(n_input=num_features, n_hidden1=224, n_hidden2=75, n_hidden3=10, n_output=len(train_data.classes))
model.fc = nn.Linear(num_features, len(train_data.classes))

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Move the model to the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def train_model(model,criterion,optimizer,trainloader,num_iter,device, len_train_data):

    model = model.to(device)
    model.train() # Set the model to training mode

    cost = []

    for epoch in range(num_iter):

        running_loss = 0.0
        train_loss = 0.0

        for i, data in enumerate(trainloader):

            # Get the inputs X and labels y for the minibatch
            inputs, labels = data[0].to(device), data[1].to(device)

            # Zero the gradients of the weights each iteration
            optimizer.zero_grad()

            # Calculate the predictions and the cost/loss
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Use autograd to calculate the gradient of the cost with respect to each weight
            loss.backward()

            # Use the optimizer to do the weights update
            optimizer.step()

            # Add the loss to running loss for the epoch
            running_loss += loss.item()

            # Update the training loss
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len_train_data
        print(f"Epoch [{epoch + 1}/{num_iter}] Train Loss: {train_loss:.4f}")

        cost.append(running_loss)
    return cost

In [ ]:
# Define the cost / loss function
criterion = nn.CrossEntropyLoss()
# Define the method of updating the weights each iteration
optimizer = optim.SGD(net.parameters(), lr=0.01)
# Number of iterations (epochs) to train
n_iter = 10
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train model
cost_path = train_model(net,criterion,optimizer,train_loader,n_iter,device,len(train_data))

# Plot the cost over training
plt.plot(cost_path)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
def test_model(model,testloader,device):
    # Turn autograd off
    with torch.no_grad():

        # Set the model to evaluation mode
        model.eval()

        # Set up lists to store true and predicted values
        y_true = []
        test_preds = []

        # Calculate the predictions on the test set and add to list
        for data in testloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            # Feed inputs through model to get raw scores
            logits = model.forward(inputs)
            # Convert raw scores to probabilities (not necessary since we just care about discrete probs in this case)
            probs = F.softmax(logits,dim=1)
            # Get discrete predictions using argmax
            preds = np.argmax(probs.cpu().numpy(),axis=1)
            test_preds.extend(preds)
            y_true.extend(labels)

        # Calculate the accuracy
        test_acc = np.sum(test_preds==y_test)/len(y_test)

    return test_acc

In [ ]:
# Test model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
acc = test_model(model,test_loader,device)
print('Test set accuracy is {:.3f}'.format(acc))

In [ ]:
# OPTION 2: Save the entire model

model_dir = 'models/'
os.makedirs(os.path.dirname(model_dir), exist_ok=True)
filename = 'multi_class_model.pt'

# Save the entire model
torch.save(net, model_dir+filename)

In [ ]:
# Define the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    # Train the model on the training set
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move the data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            # Move the data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update the test loss and accuracy
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_acc += torch.sum(preds == labels.data)

    # Print the training and test loss and accuracy
    train_loss /= len(train_data)
    test_loss /= len(test_data)
    test_acc = test_acc.double() / len(test_data)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")